## QC: Housing unit by year

###  Urbansim should match housing unit targets

In [ ]:
import os
import sys

In [ ]:
# append path to find utils module in urbansim
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
parentdir2 =  os.path.abspath(os.path.join(parentdir, os.pardir))
sys.path.append(parentdir2) # to get path to utils module

In [ ]:
import pandas as pd
import utils

In [ ]:
from sqlalchemy import create_engine
from database import get_connection_string

In [ ]:
%matplotlib inline

In [ ]:
# connect to database
db_connection_string = get_connection_string('..\..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
# get versions
versions = utils.yaml_to_dict('../../data/scenario_config.yaml', 'scenario')

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))

In [ ]:
output_sql = '''
    SELECT year_simulation,
           sum(unit_change) as unit_change
    FROM urbansim.urbansim.urbansim_lite_output urbansim_lite_output 
    WHERE  run_id =  %s
    GROUP BY year_simulation'''
output_sql = output_sql % run_id
output = pd.read_sql(output_sql,mssql_engine)

In [ ]:
units_needed_sql = '''
SELECT [yr],  [housing_units_add] as housing_units_to_add
  FROM [urbansim].[urbansim].[urbansim_target_housing_units]
  WHERE version_id = %s'''
units_needed_sql = units_needed_sql % versions['target_housing_units_version']
units_needed_df =  pd.read_sql(units_needed_sql, mssql_engine)
units_needed_df['housing_units_to_add'] = units_needed_df['housing_units_to_add'].astype(int)

In [ ]:
housing_units_by_yr = pd.merge(units_needed_df,output,\
                               left_on='yr',right_on='year_simulation')
housing_units_by_yr.drop(['year_simulation'], axis=1,inplace=True)

# QC Pass/Fail

In [ ]:
housing_units_by_yr['diff'] = housing_units_by_yr['housing_units_to_add'] - housing_units_by_yr['unit_change']
target_diff = housing_units_by_yr['diff'].sum()
if target_diff !=0:
    print('\n\nQC Fail: Housing Units Targets and Urbansim output do not match\n\n')
else:
    print('\n\nQC Pass\n\n    Urbansim output matches Housing Unit Targets\n\n')
#housing_units_by_yr

###  plot results of QC

In [ ]:
housing_units_by_yr.set_index('yr',inplace=True)
axes = housing_units_by_yr.plot(style='.-',subplots=True, figsize=(6, 6));
ptitle = 'DRAFT Housing Units By Year \n(version_id=' + \
str(versions['target_housing_units_version']) + ')'
axes.flat[0].set_title(ptitle,size=16) 
axes[0].set_ylabel('housing units')
axes[1].set_ylabel('housing units')

### tabular results of QC

In [ ]:
housing_units_by_yr